In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Preprocess the data
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))
X_train, X_test= X_train / 255.0, X_test / 255.0

In [ ]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10))


# Compile and train the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test,  y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 77s 41ms/step - loss: 0.1452 - accuracy: 0.9560 - val_loss: 0.0464 - val_accuracy: 0.9842
Epoch 2/10
1875/1875 [==============================] - 59s 31ms/step - loss: 0.0469 - accuracy: 0.9853 - val_loss: 0.0502 - val_accuracy: 0.9848
Epoch 3/10
1875/1875 [==============================] - 58s 31ms/step - loss: 0.0340 - accuracy: 0.9895 - val_loss: 0.0368 - val_accuracy: 0.9888
Epoch 4/10
1875/1875 [==============================] - 57s 31ms/step - loss: 0.0263 - accuracy: 0.9921 - val_loss: 0.0349 - val_accuracy: 0.9892
Epoch 5/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0212 - accuracy: 0.9933 - val_loss: 0.0362 - val_accuracy: 0.9898
Epoch 6/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0157 - accuracy: 0.9948 - val_loss: 0.0280 - val_accuracy: 0.9919
Epoch 7/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0137 - accuracy: 0.9957 - val_loss: 0.0353 -

In [ ]:
# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracy = []

for train_index, val_index in kfold.split(X_train):
   x_train_fold, x_val_fold = X_train[train_index], X_train[val_index]
   y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

In [ ]:
model.fit(x_train_fold, y_train_fold, epochs=5, batch_size=64, validation_data=(x_val_fold, y_val_fold))

Epoch 1/5
750/750 [==============================] - 43s 57ms/step - loss: 0.0035 - accuracy: 0.9988 - val_loss: 0.0037 - val_accuracy: 0.9986
Epoch 2/5
750/750 [==============================] - 50s 66ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0043 - val_accuracy: 0.9985
Epoch 3/5
750/750 [==============================] - 41s 55ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0031 - val_accuracy: 0.9988
Epoch 4/5
750/750 [==============================] - 42s 56ms/step - loss: 0.0018 - accuracy: 0.9995 - val_loss: 0.0110 - val_accuracy: 0.9968
Epoch 5/5
750/750 [==============================] - 41s 55ms/step - loss: 0.0070 - accuracy: 0.9976 - val_loss: 0.0069 - val_accuracy: 0.9979


In [ ]:
_, accuracy = model.evaluate(x_val_fold, y_val_fold)
fold_accuracy.append(accuracy)

375/375 [==============================] - 4s 11ms/step - loss: 0.0069 - accuracy: 0.9979


In [ ]:
# Calculate average accuracy across folds
average_accuracy = np.mean(fold_accuracy)
print(f'Average Validation Accuracy: {average_accuracy}')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

Average Validation Accuracy: 0.9979166388511658
313/313 [==============================] - 3s 9ms/step - loss: 0.0402 - accuracy: 0.9924
Test Accuracy: 0.9923999905586243
